In [ ]:
import sys

from IPython.display import Image, display
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
%cd '/content/drive/MyDrive/Inteligencia Artificial/IA - Clases de Práctica/ContenidosPorTemas/ActividadesPracticas'

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Segmentación de clientes 👩🤠🧑

El siguiente conjunto de datos consiste en información sobre el comportamiento de compra de 2,000 individuos de una determinada área al entrar en una tienda física de bienes de consumo rápido. Todos los datos se han recopilado a través de las tarjetas que utilizan al momento de pagar. Los datos han sido preprocesados y no hay valores faltantes. Además, el volumen del conjunto de datos se ha limitado y anonimizado para proteger la privacidad de los clientes.

Columnas del conjunto de datos:

- **Sex**: género.
        0: masculino,
        1: femenino.
- **Marital status**: Estado civil de un cliente.
        0: soltero,
        1: no soltero (divorciado/separado/casado/viudo).
- **Age**: Edad del cliente en años. Valor mínimo: 18, Valor máximo: 78.
- **Education**: Nivel educativo del cliente.
        0: otro/desconocido,
        1: escuela secundaria,
        2: universidad,
        3: posgrado.
- **Income**: Ingreso anual autoreportado en dólares estadounidenses del cliente. Valor mínimo: 35,832, Valor máximo: 309,364.
- **Occupation**: Categoría de la ocupación del cliente.
        0: desempleado/no calificado,
        1: empleado calificado/funcionario,
        2: administración/autónomo/empleado altamente calificado/oficial.
- **Settlement size**: Tamaño de la ciudad donde vive el cliente.
        0: ciudad pequeña,
        1: ciudad mediana,
        2: gran ciudad.

A continuación, Aplique el algoritmo de K-means para realizar una **segmentación de clientes** (**customer segmentation**), esta es una estrategia y técnica en marketing y análisis de datos que implica dividir una base de clientes en grupos más pequeños y homogéneos según características y comportamientos similares. El objetivo principal de la segmentación de clientes es comprender mejor a los diferentes grupos de clientes para poder adaptar estrategias y enfoques de marketing específicos a cada segmento.

In [ ]:
customer = pd.read_csv('1_datos/customer_data.csv')
customer.head()

> Cuando estamos aplicando kmeans a datos del mundo real usando la métrica de distancia euclídea, queremos asegurarnos que las características se miden en la misma escala por lo que es necesario escalar los datos. Utilice el escalado _MinMax_

Grafique el valor de silueta en función de _k_ y el Diagrama de Silueta para obtener un valor óptimo de _k_.

Una vez agrupadas las instancias en _k_ clusters, cree un nuevo dataframe agregando como columna adicional, las etiquetas otorgadas por el modelo de _k-means_.
Ahora se podrían filtrar los datos de cada cluster para analizar las características específicas de cada grupo. Sin embargo, se puede utilizar un modelo adicional para ayudar con esta tarea.

Utilice un *árbol de decisión* como una representación visual para la toma de decisiones basadas en las características de los clientes y asignarlos a diferentes segmentos o agrupamientos (*clusters*). Utilice el coeficiente de gini, limite la profundidad del árbol a 4 y establezca un número mínimo de 5 instancias en un nodo hoja.

**Clasifique los clientes y escriba las características de los clientes encontradas en cada cluster.**

Para clasificar los clientes, nos fijamos en la característica evaluada en cada nodo. Si se cumple lo que se está evaluando, descendemos al nodo izquierdo, de lo contrario descendemos al nodo derecho, así hasta alcanzar un nodo hoja .

- Cluster 1 -->
- Cluster 2 -->

...
